<a href="https://colab.research.google.com/github/shrikantvarma/AgenticAI/blob/main/Basic_Agent_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
import warnings

# Ignore DeprecationWarning specifically from the jupyter_client module
# warnings.filterwarnings("ignore", category=DeprecationWarning, module="jupyter_client")

# You can also ignore all DeprecationWarnings (use with caution)
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [129]:
!pip install openai duckduckgo-search



In [131]:
from openai import OpenAI
from duckduckgo_search import DDGS
import json

In [132]:
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY').strip()
client = OpenAI(api_key=openai_api_key)

In [133]:
# -------------------------------------------------------
# Define the tools (functions the agent can call)
# -------------------------------------------------------

def get_weather(city: str):
    """Return fake weather data for demo."""
    data = {
        "Paris": {"temperature": "23°C", "condition": "Sunny"},
        "London": {"temperature": "18°C", "condition": "Cloudy"},
        "Tokyo": {"temperature": "27°C", "condition": "Humid"},
    }
    return data.get(city, {"temperature": "Unknown", "condition": "Unknown"})


def calculate_expression(expression: str):
    """Safely evaluate a math expression."""
    try:
        result = eval(expression, {"__builtins__": {}})
        return {"result": result}
    except Exception as e:
        return {"error": str(e)}

def search_web_news(query: str, max_results: int = 3):
    """Use DuckDuckGo to search the web and return top news results."""
    results = []
    with DDGS() as ddgs:
        # Use the news() method for potentially more relevant news results
        for r in ddgs.news(query, max_results=max_results):
            results.append({"title": r["title"], "snippet": r["body"], "url": r["url"]}) # Note: news() returns 'url' instead of 'href'
    return results

# Map tool names → Python functions
tool_functions = {
    "get_weather": get_weather,
    "calculate_expression": calculate_expression,
    "search_web_news": search_web_news,
}

In [134]:
# -------------------------------------------------------
# Define the tool schemas (so LLM knows they exist)
# -------------------------------------------------------
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather for a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "Name of the city"},
                },
                "required": ["city"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_expression",
            "description": "Perform a basic math calculation.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Math expression like '12*(4+7)'",
                    },
                },
                "required": ["expression"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "search_web_news",
            "description": "Search the web for recent news.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Search query"},
                    "max_results": {
                        "type": "integer",
                        "description": "Number of search results to retrieve",
                    },
                },
                "required": ["query"],
            },
        },
    },
]

In [135]:
# -------------------------------------------------------
# Explicit ReAct System Prompt
# -------------------------------------------------------
SYSTEM_PROMPT = """You are an intelligent ReAct-style reasoning assistant.

Follow this loop until you can give the final answer:

Thought: reason about what to do next.
Action: if a tool is needed, call one using the exact JSON arguments.
Observation: you will receive the tool’s result.
Final Answer: only once you have enough information.

Use this format:
Thought: ...
Action: ...
Observation: ...
Thought: ...
Final Answer: ...

Always think step by step, and only call tools when necessary.
"""

In [136]:
# -------------------------------------------------------
# Core ReAct loop
# -------------------------------------------------------
import json
import re # Import the regex module

def react_agent(user_query, model="gpt-4o-mini", max_steps=5):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query},
    ]

    print(f"\n🧠 User asked: {user_query}")

    for step in range(max_steps):
        print(f"\n--- Messages being sent at Step {step+1} ---")
        for message in messages:
            print(message)
        print("----------------------------")

        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools
        )
        print(f"\n--- Response received at Step {step+1} ---")
        print(response) # Print the entire response object
        print("----------------------------")


        message = response.choices[0].message
        content = message.content or ""

        # 🧩 Print model reasoning (Thought/Action/Observation)
        if content:
            print(f"\n🧩 Model Output Step {step+1}:\n{content.strip()}")

        tool_call = None
        func_name = None
        args = None
        simulated_tool_call_id = None # Initialize simulated ID

        # Check for structured tool calls first (preferred)
        if hasattr(message, "tool_calls") and message.tool_calls:
            tool_call = message.tool_calls[0]
            func_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)
            print(f"\n🤖 Model provided structured tool call: `{func_name}` with args: {args}")
            # For structured calls, the tool_call object already has the ID
            simulated_tool_call_id = tool_call.id


        # If no structured tool calls, try parsing from content (ReAct style)
        elif "Action:" in content:
            print("\n🔍 Attempting to parse Action from content...")
            # Updated regex to handle "Action: {"key": "value"}" or "Action: tool_name({"key": "value"})"
            match_json_args = re.search(r"Action:\s*(?:(\w+)\s*)?(\{.*\})", content)
            match_func_args = re.search(r"Action:\s*(\w+)\s*\((.*?)\)", content) # Keep old regex for func(arg=val) format

            if match_json_args:
                func_name = match_json_args.group(1) # Optional function name before JSON
                args_str = match_json_args.group(2)
                try:
                    args = json.loads(args_str)
                    # If function name wasn't explicitly provided before JSON, try to infer from args if possible
                    if func_name is None and 'city' in args:
                         func_name = 'get_weather'
                         print("✅ Parsed Action: Inferred `get_weather` from args.")
                    elif func_name is None and 'query' in args:
                         func_name = 'search_web'
                         print("✅ Parsed Action: Inferred `search_web` from args.")


                    if func_name:
                         print(f"✅ Parsed Action: `{func_name}` with JSON args: {args}")
                         simulated_tool_call_id = f"parsed_call_{step}" # Create a unique ID for the parsed call

                    else:
                         print("❌ Could not determine function name from JSON action.")
                         func_name = None # Invalidate if function name can't be determined


                except json.JSONDecodeError as e:
                     print(f"❌ Failed to parse JSON arguments '{args_str}': {e}")
                     func_name = None # Invalidate the parsed action if args parsing fails
                except Exception as e:
                     print(f"❌ An unexpected error occurred during JSON parsing: {e}")
                     func_name = None

            elif match_func_args: # Fallback to old regex if JSON format not matched
                 func_name = match_func_args.group(1)
                 args_str = match_func_args.group(2)
                 try:
                     # Attempt to parse old style key=value args
                     args_dict = {}
                     if args_str:
                          arg_matches = re.findall(r'(\w+)=["\']?(.*?)["\']?(?:,\s*)?', args_str)
                          for key, value in arg_matches:
                              args_dict[key] = value
                          if not args_dict and args_str.strip():
                              args_dict['query'] = args_str.strip()

                     args = args_dict
                     print(f"✅ Parsed Action (old format): `{func_name}` with args: {args}")
                     simulated_tool_call_id = f"parsed_call_{step}" # Create a unique ID for the parsed call


                 except Exception as e:
                      print(f"❌ Failed to parse arguments (old format) '{args_str}': {e}")
                      func_name = None

            else:
                 print("❌ Could not parse Action: pattern not matched.")


        # If a tool call (structured or parsed) was identified AND a simulated ID was created
        if func_name and simulated_tool_call_id:
             if func_name in tool_functions:
                 print(f"🤖 Invoking tool `{func_name}` with args: {args}")
                 result = tool_functions[func_name](**args)
                 print(f"🧾 Observation from {func_name}: {result}")

                 # Feed observation back
                 # When parsing from content, construct a message that mimics a structured tool call
                 messages.append({
                     "role": "assistant",
                     "content": content, # Include the original content
                     "tool_calls": [{ # Add the simulated tool_calls structure
                         "id": simulated_tool_call_id,
                         "function": {
                             "name": func_name,
                             "arguments": json.dumps(args) # Arguments need to be a JSON string here
                         },
                         "type": "function"
                     }]
                 })
                 messages.append({"role": "tool", "tool_call_id": simulated_tool_call_id, "content": str(result)})
                 continue # go to next step

             else:
                 print(f"⚠️ Unknown tool: {func_name}")
                 break


        # If model gives final answer AND no tool calls (structured or parsed) were made in this step
        # The check for func_name implies no tool call was successfully identified/parsed
        if "Final Answer:" in content and not func_name:
             print(f"\n✅ Final Answer:\n{content}")
             break

        # If no tool calls (structured or parsed) and no final answer yet
        if not func_name:
             print("\n⚠️ Model did not provide a tool call or a final answer in this step.")
             break

In [137]:
react_agent("What's the weather like in Paris?")



🧠 User asked: What's the weather like in Paris?

--- Messages being sent at Step 1 ---
{'role': 'system', 'content': 'You are an intelligent ReAct-style reasoning assistant.\n\nFollow this loop until you can give the final answer:\n\nThought: reason about what to do next.\nAction: if a tool is needed, call one using the exact JSON arguments.\nObservation: you will receive the tool’s result.\nFinal Answer: only once you have enough information.\n\nUse this format:\nThought: ...\nAction: ...\nObservation: ...\nThought: ...\nFinal Answer: ...\n\nAlways think step by step, and only call tools when necessary.\n'}
{'role': 'user', 'content': "What's the weather like in Paris?"}
----------------------------

--- Response received at Step 1 ---
ChatCompletion(id='chatcmpl-CPucqiqkTcfJeN5tcdc4nILSLI3YB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Thought: I need to retrieve the current weather for Paris. \nAction: { "city": "Paris" }\nO

In [138]:
react_agent("Give me the latest AI news")


🧠 User asked: Give me the latest AI news

--- Messages being sent at Step 1 ---
{'role': 'system', 'content': 'You are an intelligent ReAct-style reasoning assistant.\n\nFollow this loop until you can give the final answer:\n\nThought: reason about what to do next.\nAction: if a tool is needed, call one using the exact JSON arguments.\nObservation: you will receive the tool’s result.\nFinal Answer: only once you have enough information.\n\nUse this format:\nThought: ...\nAction: ...\nObservation: ...\nThought: ...\nFinal Answer: ...\n\nAlways think step by step, and only call tools when necessary.\n'}
{'role': 'user', 'content': 'Give me the latest AI news'}
----------------------------

--- Response received at Step 1 ---
ChatCompletion(id='chatcmpl-CPucz50Nr5Vg75MpzhVXAiUClzdWx', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[Ch

/tmp/ipython-input-3113594215.py:26: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use time

🧾 Observation from search_web_news: [{'title': "The World's First AI-Powered Minister Tests the Future of Government", 'snippet': "An AI system named Diella has joined Albania's cabinet, testing how far governments are willing to trust machines with power.", 'url': 'https://www.msn.com/en-us/news/world/the-first-ai-powered-minister-tests-the-future-of-government/ar-AA1OefCU'}, {'title': "'It's going to be really bad': Fears over AI bubble bursting grow in Silicon Valley", 'snippet': 'In recent days, warnings of an AI bubble have come from the Bank of England, the International Monetary Fund, as well as JP Morgan boss Jamie Dimon who told the BBC "the level of uncertainty should be higher in most people\'s minds". And here, in what is often considered the tech capital of the world, concerns are growing.', 'url': 'https://www.msn.com/en-us/money/technology/its-going-to-be-really-bad-fears-over-ai-bubble-bursting-grow-in-silicon-valley/ar-AA1Ofkh7'}, {'title': "Databricks CEO says AI tale

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


--- Response received at Step 2 ---
ChatCompletion(id='chatcmpl-CPud2V5vNWwzvtn7CFUrNjcFxF5xJ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Thought: I have gathered the latest AI news articles. I will summarize them for easier consumption.\n\nFinal Answer: Here are the latest AI news articles:\n\n1. **[The World's First AI-Powered Minister Tests the Future of Government](https://www.msn.com/en-us/news/world/the-first-ai-powered-minister-tests-the-future-of-government/ar-AA1OefCU)** - An AI system named Diella has joined Albania's cabinet, testing how far governments are willing to trust machines with power.\n\n2. **['It's going to be really bad': Fears over AI bubble bursting grow in Silicon Valley](https://www.msn.com/en-us/money/technology/its-going-to-be-really-bad-fears-over-ai-bubble-bursting-grow-in-silicon-valley/ar-AA1Ofkh7)** - In Silicon Valley, warnings of a potential AI bubble are increasing, raising concerns among p

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## ReAct Agent with Tool Use

This notebook demonstrates a simple implementation of a ReAct (Reasoning and Acting) agent using the OpenAI API and external tools for interacting with the real world.

The agent follows a loop of **Thought**, **Action**, and **Observation** to process user queries.

**How it Works:**

1.  The agent receives a user query.
2.  Based on the query and a system prompt guiding the ReAct pattern, the language model (e.g., `gpt-4o-mini`) generates a **Thought** process and potentially an **Action**.
3.  If the model suggests an **Action** (a tool call), the Python code in the notebook intercepts this request.
4.  The code executes the corresponding external **Tool** function with the arguments provided by the model.
5.  The result from the tool execution (the **Observation**) is fed back into the conversation history.
6.  The updated conversation history is sent back to the language model, allowing it to consider the **Observation** and continue the loop with a new **Thought**, another **Action**, or a **Final Answer**.
7.  The loop continues until the model provides a **Final Answer** or a maximum number of steps is reached.

**Tools Included:**

*   `get_weather`: Fetches fake weather data for a given city (demonstration tool).
*   `calculate_expression`: Safely evaluates a mathematical expression.
*   `search_web_news`: Searches for recent news on the web using DuckDuckGo.

**Libraries Used:**

*   `openai`: For interacting with the OpenAI API.
*   `duckduckgo_search`: For performing web searches.
*   `json` and `re`: For parsing tool call information from the model's responses when not provided in a structured format.

This agent structure can be extended with more complex tools and refined parsing logic to handle a wider range of tasks.

# Task
Modify the `react_agent` function to limit web searches to a maximum of 5 unique queries and summarize the results if enough information is gathered.

## Modify `react agent` to track search queries

### Subtask:
Add a list within the `react_agent` function to store the unique web search queries that have been attempted.


**Reasoning**:
I need to add a list to store unique web search queries inside the `react_agent` function.



In [71]:
def react_agent(user_query, model="gpt-4o-mini", max_steps=5):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query},
    ]

    # Initialize list to store unique web search queries
    attempted_search_queries = []

    print(f"\n🧠 User asked: {user_query}")

    for step in range(max_steps):
        print(f"\n--- Messages being sent at Step {step+1} ---")
        for message in messages:
            print(message)
        print("----------------------------")

        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools
        )
        print(f"\n--- Response received at Step {step+1} ---")
        print(response) # Print the entire response object
        print("----------------------------")


        message = response.choices[0].message
        content = message.content or ""

        # 🧩 Print model reasoning (Thought/Action/Observation)
        if content:
            print(f"\n🧩 Model Output Step {step+1}:\n{content.strip()}")

        # If model triggers a tool
        if hasattr(message, "tool_calls") and message.tool_calls:
            tool_call = message.tool_calls[0]
            func_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            print(f"\n🤖 Model decided to call `{func_name}` with args: {args}")

            if func_name in tool_functions:
                result = tool_functions[func_name](**args)
                print(f"🧾 Observation from {func_name}: {result}")

                # Feed observation back
                messages.append({"role": "assistant", "content": content, "tool_calls": [tool_call]})
                messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": str(result)})
                continue
            else:
                print(f"⚠️ Unknown tool: {func_name}")
                break

        # If model gives final answer AND no tool calls are made in this step
        if "Final Answer:" in content and (not hasattr(message, "tool_calls") or not message.tool_calls):
             print(f"\n✅ Final Answer:\n{content}")
             break

        # If no tool calls and no final answer yet, something might be wrong or need more steps
        if not hasattr(message, "tool_calls") or not message.tool_calls:
             print("\n⚠️ Model did not provide a tool call or a final answer in this step.")
             break

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## Limit web search attempts

### Subtask:
Modify the loop or add logic within the loop to check if a web search is requested and if the maximum number of unique search queries (5 in this case) has been reached. If the limit is reached, prevent further web search tool calls.


**Reasoning**:
Modify the react_agent function to limit web searches to a maximum of 5 unique queries and summarize the results if enough information is gathered.



In [72]:
def react_agent(user_query, model="gpt-4o-mini", max_steps=5):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query},
    ]

    # Initialize list to store unique web search queries
    attempted_search_queries = []

    print(f"\n🧠 User asked: {user_query}")

    for step in range(max_steps):
        print(f"\n--- Messages being sent at Step {step+1} ---")
        for message in messages:
            print(message)
        print("----------------------------")

        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools
        )
        print(f"\n--- Response received at Step {step+1} ---")
        print(response) # Print the entire response object
        print("----------------------------")


        message = response.choices[0].message
        content = message.content or ""

        # 🧩 Print model reasoning (Thought/Action/Observation)
        if content:
            print(f"\n🧩 Model Output Step {step+1}:\n{content.strip()}")

        # If model triggers a tool
        if hasattr(message, "tool_calls") and message.tool_calls:
            tool_call = message.tool_calls[0]
            func_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            print(f"\n🤖 Model decided to call `{func_name}` with args: {args}")

            if func_name == 'search_web':
                query = args.get('query', '')
                if query in attempted_search_queries or len(attempted_search_queries) >= 5:
                    print(f"⚠️ Skipping web search '{query}' as query limit reached or query already attempted.")
                    # Feed a message back indicating the search was skipped
                    messages.append({"role": "assistant", "content": content, "tool_calls": [tool_call]})
                    messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": "Web search skipped: query limit reached or query already attempted."})
                    continue # Skip the tool call and move to the next step
                else:
                    attempted_search_queries.append(query)
                    print(f"Attempted searches: {attempted_search_queries}")
                    result = tool_functions[func_name](**args)
                    print(f"🧾 Observation from {func_name}: {result}")
                    messages.append({"role": "assistant", "content": content, "tool_calls": [tool_call]})
                    messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": str(result)})
                    continue


            if func_name in tool_functions:
                result = tool_functions[func_name](**args)
                print(f"🧾 Observation from {func_name}: {result}")

                # Feed observation back
                messages.append({"role": "assistant", "content": content, "tool_calls": [tool_call]})
                messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": str(result)})
                continue
            else:
                print(f"⚠️ Unknown tool: {func_name}")
                break

        # If model gives final answer AND no tool calls are made in this step
        if "Final Answer:" in content and (not hasattr(message, "tool_calls") or not message.tool_calls):
             print(f"\n✅ Final Answer:\n{content}")
             break

        # If no tool calls and no final answer yet, something might be wrong or need more steps
        if not hasattr(message, "tool_calls") or not message.tool_calls:
             print("\n⚠️ Model did not provide a tool call or a final answer in this step.")
             break


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## Add summarization logic

### Subtask:
After the main loop finishes (either by reaching the step limit or the search limit), add a check to see if any search results were obtained. If results are available, formulate a final response that summarizes the gathered information.


**Reasoning**:
I need to add logic after the loop to check for successful web search results and provide a final answer based on those results.



In [73]:
def react_agent(user_query, model="gpt-4o-mini", max_steps=5):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query},
    ]

    # Initialize list to store unique web search queries
    attempted_search_queries = []
    search_results_obtained = []

    print(f"\n🧠 User asked: {user_query}")

    for step in range(max_steps):
        print(f"\n--- Messages being sent at Step {step+1} ---")
        for message in messages:
            print(message)
        print("----------------------------")

        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools
        )
        print(f"\n--- Response received at Step {step+1} ---")
        print(response) # Print the entire response object
        print("----------------------------")


        message = response.choices[0].message
        content = message.content or ""

        # 🧩 Print model reasoning (Thought/Action/Observation)
        if content:
            print(f"\n🧩 Model Output Step {step+1}:\n{content.strip()}")

        # If model triggers a tool
        if hasattr(message, "tool_calls") and message.tool_calls:
            tool_call = message.tool_calls[0]
            func_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            print(f"\n🤖 Model decided to call `{func_name}` with args: {args}")

            if func_name == 'search_web':
                query = args.get('query', '')
                if query in attempted_search_queries or len(attempted_search_queries) >= 5:
                    print(f"⚠️ Skipping web search '{query}' as query limit reached or query already attempted.")
                    # Feed a message back indicating the search was skipped
                    messages.append({"role": "assistant", "content": content, "tool_calls": [tool_call]})
                    messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": "Web search skipped: query limit reached or query already attempted."})
                    continue # Skip the tool call and move to the next step
                else:
                    attempted_search_queries.append(query)
                    print(f"Attempted searches: {attempted_search_queries}")
                    result = tool_functions[func_name](**args)
                    print(f"🧾 Observation from {func_name}: {result}")
                    if result and result != []:
                        search_results_obtained.extend(result)
                    messages.append({"role": "assistant", "content": content, "tool_calls": [tool_call]})
                    messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": str(result)})
                    continue


            if func_name in tool_functions:
                result = tool_functions[func_name](**args)
                print(f"🧾 Observation from {func_name}: {result}")

                # Feed observation back
                messages.append({"role": "assistant", "content": content, "tool_calls": [tool_call]})
                messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": str(result)})
                continue
            else:
                print(f"⚠️ Unknown tool: {func_name}")
                break

        # If model gives final answer AND no tool calls are made in this step
        if "Final Answer:" in content and (not hasattr(message, "tool_calls") or not message.tool_calls):
             print(f"\n✅ Final Answer:\n{content}")
             break

        # If no tool calls and no final answer yet, something might be wrong or need more steps
        if not hasattr(message, "tool_calls") or not message.tool_calls:
             print("\n⚠️ Model did not provide a tool call or a final answer in this step.")
             break

    # After the loop finishes, check for search results and provide a final answer
    if search_results_obtained:
        summary = "Here is a summary of the search results:\n\n"
        for i, result in enumerate(search_results_obtained):
            summary += f"{i+1}. Title: {result.get('title', 'N/A')}\n   Snippet: {result.get('snippet', 'N/A')}\n   URL: {result.get('url', 'N/A')}\n\n"
        final_message = {"role": "assistant", "content": summary}
        messages.append(final_message)
        print(f"\n✅ Final Answer (Summarized):\n{summary}")
    else:
        final_message = {"role": "assistant", "content": "Could not find relevant information through web search."}
        messages.append(final_message)
        print("\n❌ Could not find relevant information through web search.")


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## Update loop exit condition

### Subtask:
Ensure the loop can also exit if the search query limit is reached.


**Reasoning**:
Modify the react_agent function to include a check for the number of attempted unique search queries before making a web search tool call and break the loop if the limit is reached.



In [74]:
def react_agent(user_query, model="gpt-4o-mini", max_steps=5):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_query},
    ]

    # Initialize list to store unique web search queries
    attempted_search_queries = []
    search_results_obtained = []

    print(f"\n🧠 User asked: {user_query}")

    for step in range(max_steps):
        print(f"\n--- Messages being sent at Step {step+1} ---")
        for message in messages:
            print(message)
        print("----------------------------")

        # Check if the search query limit is reached before calling the model
        if len(attempted_search_queries) >= 5:
            print("\n⚠️ Search query limit reached. Ending agent execution.")
            break # Exit the loop if the limit is reached

        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools
        )
        print(f"\n--- Response received at Step {step+1} ---")
        print(response) # Print the entire response object
        print("----------------------------")


        message = response.choices[0].message
        content = message.content or ""

        # 🧩 Print model reasoning (Thought/Action/Observation)
        if content:
            print(f"\n🧩 Model Output Step {step+1}:\n{content.strip()}")

        # If model triggers a tool
        if hasattr(message, "tool_calls") and message.tool_calls:
            tool_call = message.tool_calls[0]
            func_name = tool_call.function.name
            args = json.loads(tool_call.function.arguments)

            print(f"\n🤖 Model decided to call `{func_name}` with args: {args}")

            if func_name == 'search_web':
                query = args.get('query', '')
                if query in attempted_search_queries or len(attempted_search_queries) >= 5:
                    print(f"⚠️ Skipping web search '{query}' as query limit reached or query already attempted.")
                    # Feed a message back indicating the search was skipped
                    messages.append({"role": "assistant", "content": content, "tool_calls": [tool_call]})
                    messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": "Web search skipped: query limit reached or query already attempted."})
                    continue # Skip the tool call and move to the next step
                else:
                    attempted_search_queries.append(query)
                    print(f"Attempted searches: {attempted_search_queries}")
                    result = tool_functions[func_name](**args)
                    print(f"🧾 Observation from {func_name}: {result}")
                    if result and result != []:
                        search_results_obtained.extend(result)
                    messages.append({"role": "assistant", "content": content, "tool_calls": [tool_call]})
                    messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": str(result)})
                    continue


            if func_name in tool_functions:
                result = tool_functions[func_name](**args)
                print(f"🧾 Observation from {func_name}: {result}")

                # Feed observation back
                messages.append({"role": "assistant", "content": content, "tool_calls": [tool_call]})
                messages.append({"role": "tool", "tool_call_id": tool_call.id, "content": str(result)})
                continue
            else:
                print(f"⚠️ Unknown tool: {func_name}")
                break

        # If model gives final answer AND no tool calls are made in this step
        if "Final Answer:" in content and (not hasattr(message, "tool_calls") or not message.tool_calls):
             print(f"\n✅ Final Answer:\n{content}")
             break

        # If no tool calls and no final answer yet, something might be wrong or need more steps
        if not hasattr(message, "tool_calls") or not message.tool_calls:
             print("\n⚠️ Model did not provide a tool call or a final answer in this step.")
             break

    # After the loop finishes, check for search results and provide a final answer
    if search_results_obtained:
        summary = "Here is a summary of the search results:\n\n"
        for i, result in enumerate(search_results_obtained):
            summary += f"{i+1}. Title: {result.get('title', 'N/A')}\n   Snippet: {result.get('snippet', 'N/A')}\n   URL: {result.get('url', 'N/A')}\n\n"
        final_message = {"role": "assistant", "content": summary}
        messages.append(final_message)
        print(f"\n✅ Final Answer (Summarized):\n{summary}")
    else:
        final_message = {"role": "assistant", "content": "Could not find relevant information through web search."}
        messages.append(final_message)
        print("\n❌ Could not find relevant information through web search.")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

## Test the modified agent

### Subtask:
Run the `react_agent` function with a query that would typically trigger multiple searches to verify the new behavior.


**Reasoning**:
The `react_agent` function has been modified to limit web searches and provide a summary. The next step is to test this behavior by calling the function with a query that should trigger multiple searches.



In [75]:
react_agent("Tell me about the latest advancements in AI and their potential impact.")

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


🧠 User asked: Tell me about the latest advancements in AI and their potential impact.

--- Messages being sent at Step 1 ---
{'role': 'system', 'content': 'You are an intelligent ReAct-style reasoning assistant.\n\nFollow this loop until you can give the final answer:\n\nThought: reason about what to do next.\nAction: if a tool is needed, call one using the exact JSON arguments.\nObservation: you will receive the tool’s result.\nFinal Answer: only once you have enough information.\n\nUse this format:\nThought: ...\nAction: ...\nObservation: ...\nThought: ...\nFinal Answer: ...\n\nAlways think step by step, and only call tools when necessary.\n'}
{'role': 'user', 'content': 'Tell me about the latest advancements in AI and their potential impact.'}
----------------------------


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


--- Response received at Step 1 ---
ChatCompletion(id='chatcmpl-CPuHHTG6VE5MD4PbexWrtdMENWagt', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_QaLoZCgxjf3nCxy7FJZYurpQ', function=Function(arguments='{"query":"latest advancements in AI 2023","max_results":5}', name='search_web'), type='function')]))], created=1760290567, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=25, prompt_tokens=233, total_tokens=258, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
----------------------------

🤖 Model decided to c

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

...

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


--- Response received at Step 4 ---
ChatCompletion(id='chatcmpl-CPuHLsppJv6LBs24KHBYy6hpxGiJi', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_n9Xl4eZWNjX9qK1FKWYBZ6lb', function=Function(arguments='{"query":"AI advancements in 2023","max_results":5}', name='search_web'), type='function')]))], created=1760290571, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=24, prompt_tokens=334, total_tokens=358, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
----------------------------

🤖 Model decided to call `se

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag


--- Response received at Step 5 ---
ChatCompletion(id='chatcmpl-CPuHMVPMzU1UPmJCf5UbmNEI3oIKq', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_EiXF0VadCnZ4loQRp9d8dBOP', function=Function(arguments='{"query":"cutting edge AI technology 2023","max_results":5}', name='search_web'), type='function')]))], created=1760290572, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_560af6e559', usage=CompletionUsage(completion_tokens=26, prompt_tokens=367, total_tokens=393, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
----------------------------

🤖 Model decided to 

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

**Reasoning**:
The task requires analyzing the 'Age' column. The first step is to load the data and examine the 'Age' column to check for missing values, outliers, and the data type.

